In [1]:
from transformers import TFBertTokenizer, TFAutoModel
import tensorflow_text as text
import tensorflow as tf

from tensorflow_transform.tf_metadata import schema_utils
from tensorflow_metadata.proto import schema_pb2
from tfx.v1 import utils as io_utils
from pipeline import config
from google.cloud import aiplatform
import kubeflow_runner
import json
from datetime import datetime

2024-01-20 10:22:19.369521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 10:22:20.358519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
aiplatform.init(
    project=config.GOOGLE_CLOUD_PROJECT, location=config.GOOGLE_CLOUD_REGION
)


# Upload Schema to Google Cloud Storage

In [3]:
schema_path = config.SCHEMA_PATH
!gsutil cp schema.pbtxt {schema_path}

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Copying file://schema.pbtxt [Content-Type=application/octet-stream]...
/ [1 files][  335.0 B/  335.0 B]                                                
Operation completed over 1 objects/335.0 B.                                      


# multi-qa-MiniLM-L6-dot-v1

This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and was designed for semantic search. It has been trained on 215M (question, answer) pairs from diverse sources. For an introduction to semantic search, have a look at: SBERT.net - Semantic Search

https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-dot-v1

In [8]:
hf_pretrained = "sentence-transformers/multi-qa-MiniLM-L6-dot-v1"


# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = tf.cast(
        tf.tile(tf.expand_dims(attention_mask, -1), [1, 1, token_embeddings.shape[-1]]),
        tf.float32,
    )
    return tf.math.reduce_sum(
        token_embeddings * input_mask_expanded, 1
    ) / tf.math.maximum(tf.math.reduce_sum(input_mask_expanded, 1), 1e-9)


def build_model(hf_pretrained):
    inputs = tf.keras.Input(shape=(), dtype=tf.string, name="inputs")

    encoded_input = TFBertTokenizer.from_pretrained(hf_pretrained)(inputs)

    model_output = TFAutoModel.from_pretrained(hf_pretrained)(encoded_input)

    embeddings = mean_pooling(model_output, encoded_input["attention_mask"])

    norm_embeddings = tf.math.l2_normalize(embeddings, axis=1)

    return tf.keras.Model(name=hf_pretrained, inputs=inputs, outputs=norm_embeddings)


model = build_model(hf_pretrained)

2024-01-19 22:58:22.441466: I tensorflow_text/core/kernels/fast_bert_normalizer_model_builder.cc:211] CharacterSet built (lower_case_nfd_strip_accents=1). Trie data size (int32): 35072. Normalized string pool size (byte): 111208
2024-01-19 22:58:22.452619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 22:58:22.470492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 22:58:22.470668: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executo

# Upload model to GCS for bulk inferrer

In [5]:
def get_serve_tf_examples_fn(model, feature_spec) -> callable:
    """Returns a function that parses a serialized tf.Example."""

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        id = parsed_features["id"]
        title_body = parsed_features["title_body"]

        output = model(title_body)

        return {"id": id, "embedding": output}

    return serve_tf_examples_fn

In [6]:
schema = io_utils.parse_pbtxt_file(config.SCHEMA_PATH, schema_pb2.Schema())
feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec

signatures = {"serving_default": get_serve_tf_examples_fn(model, feature_spec)}

tf.saved_model.save(
    model, config.INFERRER_MODEL_GCS_DIR + "/Format-Serving", signatures=signatures
)

2024-01-19 19:56:09.393008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '5933' with dtype uint8 and shape [1688348]
	 [[{{node 5933}}]]
2024-01-19 19:56:11.599880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2024-01-19 19:56:12.307963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention

INFO:tensorflow:Assets written to: gs://stackoverflow-semantic-search-storage/models/multi-qa-MiniLM-L6-dot-v1-inferrer/1/model.savedmodel/Format-Serving/assets


INFO:tensorflow:Assets written to: gs://stackoverflow-semantic-search-storage/models/multi-qa-MiniLM-L6-dot-v1-inferrer/1/model.savedmodel/Format-Serving/assets


# Build and Run Pipeline for First Time

Build Docker Image for Pipeline

In [8]:
!gcloud builds submit \
    --gcs-source-staging-dir {config.GCS_BUCKET}/cloudbuild/semantic-search-pipeline/source \
    --gcs-log-dir {config.GCS_BUCKET}/cloudbuild/semantic-search-pipeline/logs \
    --tag {config.PIPELINE_IMAGE}

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Creating temporary tarball archive of 65 file(s) totalling 206.4 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/ivan/.config/gcloud/logs/2024.01.19/20.04.40.430849.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://stackoverflow-semantic-search-storage/cloudbuild/semantic-search-pipeline/source/1705669483.250319-121dae9e326748e9a2e5e930c38d864b.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/stackoverflow-semantic-search/locations/global/builds/8acf5fb3-92ec-41b1-9cc6-9b310c2a4afb].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/8acf5fb3-92ec-41b1-9cc6-9b310c2a4afb?project=500376812545 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------

Compile and run pipeline

In [4]:
pipeline_definition_file = kubeflow_runner.run()
pipeline_definition_file

'gs://stackoverflow-semantic-search-storage/semantic-search-pipeline.json'

In [5]:
QUERY = "SELECT id, CONCAT(title, '\\n', LEFT(body, 100)) as title_body, tags FROM bigquery-public-data.stackoverflow.posts_questions WHERE creation_date BETWEEN '{}' AND '{}' AND score > 0 ORDER BY creation_date"

date_start = datetime(day=1, month=1, year=2020).date()
date_end = datetime(day=1, month=8, year=2022).date()
bigquery_input_config = {
        "splits": [
            {
                "name": "single_split",
                "pattern": QUERY.format(str(date_start), str(date_end)),
            }
        ]
    }
bigquery_input_config

{'splits': [{'name': 'single_split',
   'pattern': "SELECT id, CONCAT(title, '\\n', LEFT(body, 100)) as title_body, tags FROM bigquery-public-data.stackoverflow.posts_questions WHERE creation_date BETWEEN '2020-01-01' AND '2022-08-01' AND score > 0 ORDER BY creation_date"}]}

In [6]:
pipeline = aiplatform.PipelineJob(
    display_name="semantic-search-pipeline",
    template_path=pipeline_definition_file,
    parameter_values={
        "bigquery_input_config": json.dumps(bigquery_input_config),
        "update_index": "",
    },
)

pipeline.submit(service_account=config.SERVICE_ACCOUNT)

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/500376812545/locations/asia-southeast1/pipelineJobs/semantic-search-pipeline-20240119202944


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/500376812545/locations/asia-southeast1/pipelineJobs/semantic-search-pipeline-20240119202944


To use this PipelineJob in another session:


INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:


pipeline_job = aiplatform.PipelineJob.get('projects/500376812545/locations/asia-southeast1/pipelineJobs/semantic-search-pipeline-20240119202944')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/500376812545/locations/asia-southeast1/pipelineJobs/semantic-search-pipeline-20240119202944')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/asia-southeast1/pipelines/runs/semantic-search-pipeline-20240119202944?project=500376812545


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/asia-southeast1/pipelines/runs/semantic-search-pipeline-20240119202944?project=500376812545


# Upload model for Serving Web Application

In [9]:
SERVING_MODEL_GCS_DIR = (
    f"{config.GCS_BUCKET}/models/multi-qa-MiniLM-L6-dot-v1/1/model.savedmodel"
)
model.save(SERVING_MODEL_GCS_DIR)

2024-01-19 22:58:36.593627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2024-01-19 22:58:37.294148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2024-01-19 22:58:37.354277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_ids_1' with dtype int32 and shape [?,?]
	 [[{{n

INFO:tensorflow:Assets written to: gs://stackoverflow-semantic-search-storage/models/multi-qa-MiniLM-L6-dot-v1/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: gs://stackoverflow-semantic-search-storage/models/multi-qa-MiniLM-L6-dot-v1/1/model.savedmodel/assets


Upload and Deploy Model to an Endpoint for Cloud Run Service

In [12]:
model = aiplatform.Model.upload(
    display_name="multi-qa-MiniLM-L6-dot-v1",
    artifact_uri=SERVING_MODEL_GCS_DIR,
    serving_container_image_uri="asia-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.2-11:latest",
    location=config.GOOGLE_CLOUD_REGION,
)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/500376812545/locations/asia-southeast1/models/5912566603760795648/operations/7202824807258259456


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/500376812545/locations/asia-southeast1/models/5912566603760795648/operations/7202824807258259456


Model created. Resource name: projects/500376812545/locations/asia-southeast1/models/5912566603760795648@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/500376812545/locations/asia-southeast1/models/5912566603760795648@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/500376812545/locations/asia-southeast1/models/5912566603760795648@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/500376812545/locations/asia-southeast1/models/5912566603760795648@1')


In [7]:
model = aiplatform.Model.list(location=config.GOOGLE_CLOUD_REGION)[0]
model

resource name: projects/500376812545/locations/asia-southeast1/models/5912566603760795648

In [8]:
endpoint = model.deploy(
    deployed_model_display_name="multi-qa-MiniLM-L6-dot-v1",
    traffic_split={"0": 100},
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=1,
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
    service_account=config.SERVICE_ACCOUNT,
)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672/operations/1475934931103252480


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672/operations/1475934931103252480


Endpoint created. Resource name: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672


...

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672


Deploy Endpoint model backing LRO: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672/operations/6236239737233866752


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672/operations/6236239737233866752


Endpoint model deployed. Resource name: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/500376812545/locations/asia-southeast1/endpoints/2452869704079900672


# Build Vector Search Index using Pipeline Run Result Artifact

In [3]:
embeddings_uri = aiplatform.Artifact.list(
    filter='schema_title="artifacts.FeatureVector"'
)[0].uri

In [15]:
index_metadata = {
    "contentsDeltaUri": embeddings_uri,
    "config": {
        "dimensions": 384,
        "approximateNeighborsCount": 150,
        "distanceMeasureType": "DOT_PRODUCT_DISTANCE",
        "shardSize": "SHARD_SIZE_SMALL",
        "algorithm_config": {
            "treeAhConfig": {
                "leafNodeEmbeddingCount": 500,
                "leafNodesToSearchPercent": 7,
            }
        },
    },
}

INDEX_METADATA_FILE = "index_metadata.json"

with open(INDEX_METADATA_FILE, "w") as outfile:
    json.dump(index_metadata, outfile)

Build Index

In [16]:
!gcloud ai indexes create \
  --metadata-file={INDEX_METADATA_FILE} \
  --display-name="semantic-search-index" \
  --project={config.GOOGLE_CLOUD_PROJECT} \
  --region={config.GOOGLE_CLOUD_REGION}

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Using endpoint [https://asia-southeast1-aiplatform.googleapis.com/]
The create index operation [projects/500376812545/locations/asia-southeast1/indexes/3631924401055727616/operations/1312116494657650688] was submitted successfully.

You may view the status of your operation with the command

  $ gcloud ai operations describe 1312116494657650688 --index=3631924401055727616 [--project=stackoverflow-semantic-search]


In [4]:
index = aiplatform.MatchingEngineIndex.list()[0]
index

resource name: projects/500376812545/locations/asia-southeast1/indexes/3631924401055727616

Create Endpoint

In [5]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="Semantic Search Index Endpoint",
    description="Index Endpoint for Stackoverflow Semantic Search",
    network=config.VPC_NETWORK,
)

Creating MatchingEngineIndexEndpoint


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint


Create MatchingEngineIndexEndpoint backing LRO: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248/operations/5749850977477853184


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248/operations/5749850977477853184


MatchingEngineIndexEndpoint created. Resource name: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248


To use this MatchingEngineIndexEndpoint in another session:


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:


index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248')


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248')


Deploy Index to the Endpoint

In [6]:
DEPLOYED_INDEX_ID = "semantic_search_index"

deployed_index = index_endpoint.deploy_index(
    index=index,
    deployed_index_id=DEPLOYED_INDEX_ID,
    machine_type="e2-standard-2",
    min_replica_count=1,
    max_replica_count=1,
)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248


Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248/operations/1680848714148610048


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248/operations/1680848714148610048


MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/500376812545/locations/asia-southeast1/indexEndpoints/996210311323189248


# Create GCP Pub/Sub Topic as a Trigger to Cloud Function

In [5]:
PUBSUB_TOPIC = f"trigger-{config.PIPELINE_NAME}"

In [7]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Created topic [projects/semantic-search-401703/topics/trigger-post-similarity-pipeline].


# Create and Deploy Cloud Function to Trigger Pipeline

In [3]:
ENV_VARS=f"""\
PROJECT={config.GOOGLE_CLOUD_PROJECT},\
REGION={config.GOOGLE_CLOUD_REGION},\
GCS_PIPELINE_FILE_LOCATION={pipeline_definition_file},\
PIPELINE_NAME={config.PIPELINE_NAME},\
SERVICE_ACCOUNT={config.SERVICE_ACCOUNT}
"""

!echo {ENV_VARS}

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
PROJECT=semantic-search-401703,REGION=asia-southeast1,GCS_PIPELINE_FILE_LOCATION=gs://semantic-search-401703-storage/post-similarity-pipeline.json,PIPELINE_NAME=post-similarity-pipeline,SERVICE_ACCOUNT=svc-acc@semantic-search-401703.iam.gserviceaccount.com


In [21]:
!gcloud functions deploy trigger-pipeline \
    --region={config.GOOGLE_CLOUD_REGION} \
    --run-service-account={config.SERVICE_ACCOUNT} \
    --service-account={config.SERVICE_ACCOUNT} \
    --max-instances=1 \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python38 \
    --gen2 \
    --source=cloud_function \
    --entry-point=trigger_pipeline \
    --stage-bucket={config.GCS_BUCKET}/cloudfunction \
    --update-env-vars={ENV_VARS} \

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Preparing function...done.                                                     
Deploying function...                                                          
  . [Build]                                                                    
  . [Service]                                                                  
  . [Trigger]                                                                  
  . [ArtifactRegistry]                                                         
  . [Healthcheck]                                                              
  . [Triggercheck]                                                             
  Deploying function...                                                        






⠛ Deploying function...                                                        






⠹ Deploying function...                                               

# Create Cloud Scheduler Job to Run Pipeline Periodically

In [18]:
SCHEDULER_JOB_NAME = "index_update"

In [26]:
!gcloud scheduler jobs create pubsub {SCHEDULER_JOB_NAME} --schedule "0 * * * 0" --topic {PUBSUB_TOPIC} --location {config.GOOGLE_CLOUD_REGION} --message-body run

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
name: projects/semantic-search-401703/locations/asia-southeast1/jobs/index_update
pubsubTarget:
  data: cnVu
  topicName: projects/semantic-search-401703/topics/trigger-post-similarity-pipeline
retryConfig:
  maxBackoffDuration: 3600s
  maxDoublings: 16
  maxRetryDuration: 0s
  minBackoffDuration: 5s
schedule: '*/1 * * * *'
state: ENABLED
timeZone: Etc/UTC
userUpdateTime: '2023-10-12T06:52:34Z'


# Delete GCP Resources

In [ ]:
endpoint.delete(force=True)
deployed_index.delete(force=True)

In [ ]:
!gcloud pubsub topics delete {PUBSUB_TOPIC}

In [28]:
!gcloud functions delete trigger-pipeline --region {config.GOOGLE_CLOUD_REGION} --quiet --gen2

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Preparing function...done.                                                     
Deleting function...                                                           
  . [Artifact Registry]                                                        
  . [Service]                                                                  
  . [Trigger]                                                                  
  Deleting function...                                                         



⠛ Deleting function...                                                         



⠹ Deleting function...                                                         


  ⠹ [Trigger]                                                                  
⠼ Deleting function...                                                         


  ⠼ [Trigger] Deleting Eventarc trigger...                              

In [27]:
!gcloud scheduler jobs delete {SCHEDULER_JOB_NAME} --location {config.GOOGLE_CLOUD_REGION} --quiet

/bin/bash: /home/ivan/miniconda3/envs/tfx/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Deleted job [index_update].
